In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from bound_funcs import *
from utils import *
from dgp import *



def e1(dgp, XU, Z):
    norm = 1/(2*np.sqrt(XU.shape[1]))
    return sigmoid(norm * (( dgp['e1_coeffs'] * XU).sum(axis=1) + dgp['beta_zd'] * Z))

def mu(dgp, coeffs, XU, Z, d):
    
    if d == 1:
        coeffs = dgp[]
    
    norm = 1/(2*np.sqrt(XU.shape[1]))
    return sigmoid(norm * ((coeffs * XU).sum(axis=1) + dgp['beta_zy'] * Z))

def check_bounds(data, Vpf_down, Vpf_up):
    '''Given observational data (Y,D,T) ~ p() compute bounds across measures of interest.'''

    Y, D, T = data['Y'], data['D'], data['T']
    v = np.zeros((2,2,2))

    for y in range(2):
        for d in range(2):
            for t in range(2):
                v[y,t,d] = ((Y==y) & (D==d) & (T==t)).mean()

    metrics = ['m_y=1', 'm_y=0', 'm_a=0', 'm_a=1', 'm_u']
    u = np.array([[1,0], [0, 1]])

    for metric in metrics:

        R_oracle = oracle_regret(v, u, metric)
        Rs_down, Rs_up = standard_bounds(v, Vpf_down, Vpf_up, u, metric)
        Rd_down, Rd_up = delta_bounds(v, Vpf_down, Vpf_up, u, metric)

        print(f'metric: {metric}')
        print(f'Standard bounds [{Rs_down:.3}, {Rs_up:.3}]')
        print(f'Delta bounds: [{Rd_down:.3}, {Rd_up:.3}]')
        print(f'Oracle: {R_oracle:.4}')
        print()

2.5

In [69]:
Dx, Du = 3, 10
nD = Dx+Du

# Measured and unmeasured covariate loadings
e1_coeffs = 2*np.random.rand(nD) - 1
z_coeffs = 2*np.random.rand(nD) - 1
w_coeffs = 2*np.random.rand(nD) - 1
mu1_coeffs = 2*np.random.rand(nD) - 1
mu0_coeffs = 2*np.random.rand(nD) - 1



dgp = {
    'N': 5000,
    'Dx': Dx,
    'Du': Du,
    'nz': 10,                # Number of finite pre-treatment values
    'nw': 10,                # Number of finite post-treatment values
    'beta_zd': 2,            # Z -> D loading (=0 ==> relevance is violated)
    'beta_zy': 0,            # Z -> Y loading (=0 ==> exclusion restriction is satisfied)
    'e1_coeffs': e1_coeffs,
    'z_coeffs': z_coeffs,
    'w_coeffs': w_coeffs,
    'mu1_coeffs': mu1_coeffs,
    'mu0_coeffs': mu0_coeffs,
    'lambda': 2,
    'assumption': 'MSM'
}

# Measured and unmeasured covariate loadings
e1_coeffs = 2*np.random.rand(nD) - 1
z_coeffs = 2*np.random.rand(nD) - 1
w_coeffs = 2*np.random.rand(nD) - 1
mu1_coeffs = 2*np.random.rand(nD) - 1
mu0_coeffs = 2*np.random.rand(nD) - 1

if dgp['assumption'] == 'MSM':
    
    dgp['lambda_star'] = np.random.uniform(1/dgp['lambda'], dgp['lambda'])
    

     
    


# Out here there should be some sort of DGP prep function that assigns the coeffs based on the causal assumptions.
# Manipulating inside sampling function less clean

dgp_assumption = 'TP'


def generate_data(dgp):
    
    # Co-variate information
    N, Dx, Du = dgp['N'], dgp['Dx'], dgp['Du']
    nD = Dx+Du
    
    # Proxy information
    nz, nw, beta_zd, beta_zy = dgp['nz'], dgp['nw'], dgp['beta_zd'], dgp['beta_zy']
    
    e1_coeffs = dgp['e1_coeffs']
    z_coeffs = dgp['z_coeffs']
    w_coeffs = dgp['w_coeffs']
    mu1_coeffs = dgp['mu1_coeffs']
    mu0_coeffs = dgp['mu0_coeffs']
    
    norm = 1/(2*np.sqrt(nD))
    T = np.random.binomial(1,.35*np.ones(N))
    
    # Sample measured and unmeasured confounders
    mean, cov = np.zeros(nD), np.eye(nD)
    XU = np.random.multivariate_normal(mean, cov, N)

    # Zero out these terms if the IV unconfoundedness assumption is satisfied
    if dgp_assumption == 'IV':
        print('zero z coefs')
        z_coeffs[Dx:] = 0

    # Compute the probability distribution for Z
    prob_Z = np.exp(z_coeffs*XU)
    prob_Z = prob_Z / np.sum(prob_Z.sum(axis=1))  
    weights = np.random.rand(nD, nz)
    logits = np.dot(XU, weights)
    pZ = softmax(logits)

    # Sample instrument values
    Z = np.argmax(np.array([np.random.multinomial(1, p) for p in pZ]), axis=1)

    # Treatment propensity is a function of X, U and Z. Beta is a scaling factor.    
    pD = e1(dgp, XU, Z)
    D = np.random.binomial(1, pD)

    # Compute the probability distribution for W
    # prob_W = np.exp(w_coeffs*XU)
    # prob_W = prob_W / np.sum(prob_W.sum(axis=1))  
    # weights = np.random.rand(nD, nW)
    # logits = np.dot(XU, weights)
    # pW = softmax(logits)
    # W = np.argmax(np.array([np.random.multinomial(1, p) for p in pW]), axis=1)
    
    p_mu_1 = mu(dgp, dgp['mu1_coeffs'], XU, Z)
    
    if dgp['assumption'] == 'MSM':
        p_mu_0 = dgp['lambda_star'] * p_mu_1
    
    else:
        p_mu_0 = mu(dgp, dgp['mu0_coeffs'], XU, Z)
    
    p_mu = pD*p_mu_1 + (1-pD)*p_mu_0

    Y = np.random.binomial(1, p_mu)


    return {
        'Y': Y,
        'D': D,
        'XU': XU,
        'Z': Z,
        'T': T
    }




In [72]:
dgp

{'N': 5000,
 'Dx': 3,
 'Du': 10,
 'nz': 10,
 'nw': 10,
 'beta_zd': 2,
 'beta_zy': 0,
 'e1_coeffs': array([-0.38423103, -0.17080017,  0.39776127, -0.5226944 ,  0.1991525 ,
         0.12916137, -0.97893472,  0.35488075, -0.28664397,  0.55518209,
         0.0286428 ,  0.5341621 , -0.16682049]),
 'z_coeffs': array([ 0.61138496,  0.96278481,  0.02573141,  0.83961893,  0.54028065,
         0.39483237, -0.36924444,  0.35752086, -0.27201822,  0.73380494,
         0.48690683, -0.6104928 , -0.63428146]),
 'w_coeffs': array([-0.5181226 , -0.66281399, -0.78760784,  0.98166515,  0.56112596,
        -0.44959156,  0.42939163, -0.8846204 , -0.39441318, -0.05480092,
        -0.69466732,  0.98277214,  0.40163069]),
 'mu1_coeffs': array([-0.15034936, -0.37835128,  0.1123084 ,  0.04581665, -0.38378139,
         0.77823759, -0.37511603,  0.28701363, -0.9623329 ,  0.42259818,
        -0.09581069, -0.2139295 , -0.69944138]),
 'mu0_coeffs': array([ 0.73208493,  0.70935961,  0.41417304,  0.67018546,  0.3749379

In [74]:


# data = generate_data(dgp)

# Vpf_down, Vpf_up = compute_iv_bounds(dgp, data)
# check_bounds(data, Vpf_down, Vpf_up)
# print()
# Vpf_down, Vpf_up = compute_na_bounds(dgp, data)
# check_bounds(data, Vpf_down, Vpf_up)
# print()

dgp['lambda'] = 1.5
Vpf_down, Vpf_up = compute_msm_bounds(dgp, data)
check_bounds(data, Vpf_down, Vpf_up)


##### Note for next time working on this. 
###   To get the tc proxy ID to work, we need to be able to get different values of py for varying z's
###   Need to be able to connect through U most likely
#####
# Vpf_down, Vpf_up = compute_tp_bounds(dgp, data)
# check_bounds(data, Vpf_down, Vpf_up)

metric: m_y=1
Standard bounds [-0.529, -0.294]
Delta bounds: [-0.487, -0.339]
Oracle: -0.4017

metric: m_y=0
Standard bounds [-0.513, -0.254]
Delta bounds: [-0.466, -0.305]
Oracle: -0.4357

metric: m_a=0
Standard bounds [-0.356, 0.189]
Delta bounds: [-0.249, 0.0823]
Oracle: 0.02611

metric: m_a=1
Standard bounds [-0.0441, 0.0644]
Delta bounds: [-0.0441, 0.0644]
Oracle: 0.01645

metric: m_u
Standard bounds [-0.148, 0.0669]
Delta bounds: [-0.0773, -0.00393]
Oracle: -0.025



In [67]:
Vpf_down, Vpf_up = compute_msm_bounds(dgp, data)
check_bounds(data, Vpf_down, Vpf_up)



KeyError: 'lambda'

In [66]:


def compute_msm_bounds(dgp, data):
    
    XU, Z, T = data['XU'], data['Z'], data['T']
    
    p_mu_1 = mu(dgp, dgp['mu1_coeffs'], XU, Z)
    mu_up = dgp['lambda'] * p_mu_1
    mu_down = (1/dgp['lambda']) * p_mu_1
    
    v110_up = (T * mu_up * p_e0).mean()
    v100_up = ((1-T) * mu_up * p_e0).mean()
    v110_down = (T * mu_down * p_e0).mean()
    v100_down = ((1-T) * mu_down * p_e0).mean()
    
    Vpf_down[0,1] = v110_down
    Vpf_down[1,1] = v110_down
    Vpf_down[0,0] = v100_down
    Vpf_down[1,0] = v100_down

    Vpf_up[0,1] = v110_up
    Vpf_up[1,1] = v110_up
    Vpf_up[0,0] = v100_up
    Vpf_up[1,0] = v100_up 
    
    return Vpf_down, Vpf_up

    

def compute_na_bounds(dgp, data):
    
    Y, D, T = data['Y'], data['D'], data['T']

    Vpf_down, Vpf_up = np.zeros((2,2)), np.zeros((2,2))
    
    Vpf_up[0,1] = ((D==0) & (T==1)).mean()
    Vpf_up[1,1] = ((D==0) & (T==1)).mean()
    Vpf_up[0,0] = ((D==0) & (T==0)).mean()
    Vpf_up[1,0] = ((D==0) & (T==0)).mean()
    
    return Vpf_down, Vpf_up


def compute_tp_bounds(dgp, data):
    XU, Z, T = data['XU'], data['Z'], data['T']

    mu_z = np.zeros((dgp['nz'], dgp['N']))

    p_e0 = 1 - e1(dgp, XU, Z)

    # Compute upper and lower bounds on mu(a,x)
    for z in range(dgp['nz']):

        # Assert: This probability should vary across levels of z, 
        # even with exclusion restriction (beta_zy=0) satisfied
        # This is because of the effects of confouners
        mu_z[z] = mu(dgp, dgp['mu1_coeffs'], XU, z)


    mu_down = mu_z.min(axis=0)
    mu_up = mu_z.max(axis=0)

    v110_up = (T * mu_up * p_e0).mean()
    v100_up = ((1-T) * mu_up * p_e0).mean()
    v110_down = (T * mu_down * p_e0).mean()
    v100_down = ((1-T) * mu_down * p_e0).mean()

    Vpf_down = np.zeros((2,2))
    Vpf_up = np.zeros((2,2))

    Vpf_down[0,1] = v110_down
    Vpf_down[1,1] = v110_down
    Vpf_down[0,0] = v100_down
    Vpf_down[1,0] = v100_down

    Vpf_up[0,1] = v110_up
    Vpf_up[1,1] = v110_up
    Vpf_up[0,0] = v100_up
    Vpf_up[1,0] = v100_up 


    

def compute_iv_bounds(dgp, data):
    
    XU, Z, T = data['XU'], data['Z'], data['T']

    mu_down_z = np.zeros((dgp['nz'], dgp['N']))
    mu_up_z = np.zeros((dgp['nz'], dgp['N']))
    
    #TODO: here we are fitting functions based on unobservables so the U terms should be zeroed for e1, mu_1 
    p_e1 = e1(dgp, XU, Z)
    p_mu_1 = mu(dgp, dgp['mu1_coeffs'], XU, Z)

    # Compute upper and lower bounds on mu(a,x)
    for z in range(dgp['nz']):
        e1_z =  e1(dgp, XU, z)
        e0_z = 1-e1_z

        mu_down_z[z] = e1_z*p_mu_1
        mu_up_z[z] = e0_z + e1_z*p_mu_1

    mu_down = mu_down_z.max(axis=0)
    mu_up = mu_up_z.min(axis=0)
    
    v110_up = (T * (mu_up - p_mu_1 * p_e1)).mean()
    v100_up = ((1-T) * (mu_up - p_mu_1 * p_e1)).mean()

    v110_down = (T * (mu_down - p_mu_1 * p_e1)).mean()
    v100_down = ((1-T) * (mu_down - p_mu_1 * p_e1)).mean()
    
    Vpf_down = np.zeros((2,2))
    Vpf_up = np.zeros((2,2))
    
    Vpf_down[0,1] = v110_down
    Vpf_down[1,1] = v110_down
    Vpf_down[0,0] = v100_down
    Vpf_down[1,0] = v100_down
    
    Vpf_up[0,1] = v110_up
    Vpf_up[1,1] = v110_up
    Vpf_up[0,0] = v100_up
    Vpf_up[1,0] = v100_up  
    
    return Vpf_down, Vpf_up



    

# Vpf_down = np.zeros((2,2))
# Vpf_up = np.zeros((2,2))
# v = np.zeros((2,2,2))

# for y in range(2):
#     for d in range(2):
#         for t in range(2):
#             v[y,t,d] = ((Y==y) & (D==d) & (T==t)).mean()


# Vpf_up[0,1] = v110_up
# Vpf_up[1,1] = v110_up
# Vpf_up[0,0] = v100_up
# Vpf_up[1,0] = v100_up            

# Vpf_down[0,1] = v110_down
# Vpf_down[1,1] = v110_down
# Vpf_down[0,0] = v100_down
# Vpf_down[1,0] = v100_down


    
# metrics = ['m_y=1', 'm_y=0', 'm_a=0', 'm_a=1', 'm_u']
# u = np.array([[1,0], [0, 1]])

# for metric in metrics:
#     R_oracle = oracle_regret(v, u, metric)

#     Rs_down, Rs_up = standard_bounds(v, Vpf_down, Vpf_up, u, metric)
#     Rd_down, Rd_up = delta_bounds(v, Vpf_down, Vpf_up, u, metric)

#     print(f'metric: {metric}')
#     print(f'Standard bounds [{Rs_down:.3}, {Rs_up:.3}]')
#     print(f'Delta bounds: [{Rd_down:.3}, {Rd_up:.3}]')
#     print(f'Oracle: {R_oracle:.4}')
#     print()

## Now, given these nuisance functions, we'd like to estimate Vpf.

## Test computing bounds from the sampled joint distribution

In [11]:
data = {
    'Y': Y, 
    'D': D, 
    'T': T
}

check_p_bounds(data)

NameError: name 'Y' is not defined

## Scratch stuff


In [18]:
# Creating the Logistic Regression model
model = LogisticRegression()

# Fitting the model
model.fit(XU, D)

# Making predictions and evaluating the model
predictions = model.predict_proba(XU)
accuracy = accuracy_score(D, predictions)
print(f"Accuracy: {accuracy}")
predictions

NameError: name 'XU' is not defined